In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dask
from sklearn.model_selection import StratifiedKFold
import joblib
from dask.distributed import Client
from sklearn.model_selection import cross_val_score
!pip install dask-ml
from sklearn.linear_model import LogisticRegression
import dask_ml.model_selection as dcv
#import dask_ml.linear_model as LogisticRegression
from sklearn.utils import resample
!pip install -U imbalanced-learn
import imblearn



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.5/148.5 kB 357.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 489.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 1.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: joblib
    Found existing installation: joblib 1.0.1
    Uninstalling joblib-1.0.1:
      Successfully uninstalled joblib-1.0.1
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.9.0
    Uninstalling imbalanced-learn-0.9.0:
      Successfully uninstalled imbalanced-learn-0.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pdpbox 0.2.1 requires matplotlib==3.1.1, but you have matplotlib 3.5.3 which is incompatible.
pandas-profiling 3.1.0 requires joblib~=1.0.1, but you have joblib 1.2.0 which is incompatible.
pandas-profiling 3.1.

In [2]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier



In [3]:
client = Client(processes=False)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://172.19.2.2:8787/status,
Dashboard: http://172.19.2.2:8787/status,Workers: 1
Total threads: 4,Total memory: 31.36 GiB
Status: running,Using processes: False
Comm: inproc://172.19.2.2/27/1,Workers: 1
Dashboard: http://172.19.2.2:8787/status,Total threads: 4
Started: Just now,Total memory: 31.36 GiB
Comm: inproc://172.19.2.2/27/4,Total threads: 4
Dashboard: http://172.19.2.2:38729/status,Memory: 31.36 GiB
Nanny: None,


In [4]:
df = pd.read_csv("/kaggle/input/datasetmodified/myTrain.csv")
dft = pd.read_csv("/kaggle/input/datasetmodified/myTest.csv")
LabelsT = pd.read_csv("/kaggle/input/datasetmod/labels.csv")

In [1]:
print(df.shape)
print(dft.shape)
print(LabelsT.shape)

NameError: name 'df' is not defined

In [10]:

# LabelsT = np.ravel(LabelsT)

In [40]:
dfm = df.sample(n=80000)
LabelsTm= LabelsT.sample(n=80000)
print(dfm.shape)
print(dft.shape)
print(LabelsTm.shape)

(80000, 29)
(1000000, 29)
(80000, 1)


In [6]:
# skfold=StratifiedKFold(n_splits=5)
# model = LogisticRegression(solver='sag',max_iter = 1000, n_jobs=-1)

# # with joblib.parallel_backend("dask"):
# scores = cross_val_score(model,df,LabelsT,cv=skfold)
    
# print(np.mean(scores))    


In [2]:
# parameters = {'class_weight':['balanced'],'solver':['sag','saga'],'penalty':['l2'],'warm_start':['true']}
# lr = LogisticRegression()
# clf = dcv.RandomizedSearchCV(lr,parameters)
# clf.fit(df,LabelsT)

In [6]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek()
X_smt, y_smt = smt.fit_resample(dfm, LabelsTm)

/opt/conda/lib/python3.7/site-packages/imblearn/over_sampling/_smote/base.py:340: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  FutureWarning,


In [5]:
# print(X_smt.shape)
# y_smt = np.ravel(y_smt)
kfold_validation=StratifiedKFold(10)

In [43]:
parameters = {'solver':['sag','saga'],'penalty':['l2','none'],'warm_start':['true'],'C':[1.0,0.8,0.1],'max_iter':[1000]}
lr = LogisticRegression()
clf = RandomizedSearchCV(lr,parameters,cv = kfold_validation)
search = clf.fit(dfm, LabelsTm)
search.best_params_

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

{'warm_start': 'true',
 'solver': 'sag',
 'penalty': 'l2',
 'max_iter': 1000,
 'class_weight': 'balanced',
 'C': 0.1}

In [44]:
search.best_score_

0.4775125

In [45]:
search.score(df,LabelsT)

0.49568887834350606

In [ ]:
#Training Model based on hyperparameter tuning

In [54]:
from sklearn.model_selection import train_test_split
X_train_val, X_test_val, y_train_val, y_test_val = train_test_split(df, LabelsT, test_size = .2, stratify=LabelsT)

TypeError: train_test_split() got an unexpected keyword argument 'cv'

In [47]:
y_train_val = np.ravel(y_train_val)
y_test_val = np.ravel(y_test_val)

In [49]:
lr = LogisticRegression(solver='sag', max_iter=10000, n_jobs=-1,class_weight='balanced',penalty='l2',C=0.1)
lr.fit(X_train_val,y_train_val)

LogisticRegression(C=0.1, class_weight='balanced', max_iter=10000, n_jobs=-1,
                   solver='sag')

In [50]:
y_val_preds = lr.predict(X_test_val)  

from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score, confusion_matrix, precision_recall_curve, PrecisionRecallDisplay

print("Accuracy: {},Precision: {}, Recall: {}, F2 Score: {}".format(accuracy_score(y_test_val, y_val_preds),precision_score(y_test_val, y_val_preds), recall_score(y_test_val, y_val_preds), fbeta_score(y_test_val, y_val_preds, beta=2.0)))


Accuracy: 0.9122092418543662,Precision: 0.6896156094576064, Recall: 0.9430771606321882, F2 Score: 0.8785003018271563


In [51]:
y_val_preds = lr.predict(dft) 
y_val_preds.shape

(1000000,)

In [52]:
my_df = pd.DataFrame(y_val_preds)
my_df.to_csv('logis5.csv',index= True)

In [55]:
#Decision tree Classifier
parameters = {'criterion':['gini', 'entropy'],'splitter':['best','random'],'max_depth':[10,5],
              'max_features':['auto','sqrt','log2']}
dc = DecisionTreeClassifier()
clf =  RandomizedSearchCV(dc,parameters,cv=kfold_validation)
search2 = clf.fit(df,LabelsT)
search2.best_params_

{'splitter': 'best',
 'max_features': 'auto',
 'max_depth': 10,
 'criterion': 'entropy'}

In [56]:
search2.best_score_

0.9816232062473536

In [57]:
search2.score(df,LabelsT)

0.9872898261198797

In [73]:
y_val = search2.scoring
y_val


In [65]:
my_df = pd.DataFrame(ynew)
my_df.to_csv('dt2.csv',index= True)

In [78]:
#SGD Classifier
parameters = {'tol':[1e-3,1e-4],'loss':['log','log-loss','hinge','perceptron'],'penalty':['l2','l1','none'],
              'alpha':[0.1,0.5,2,0.001],'shuffle':[True]}
est = SGDClassifier()
clf = RandomizedSearchCV(est,parameters,cv=kfold_validation)
search3 = clf.fit(df,LabelsT)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [79]:
search3.best_params_

{'tol': 0.0001,
 'shuffle': True,
 'penalty': 'none',
 'loss': 'perceptron',
 'alpha': 0.001}

In [80]:
search3.best_score_

0.9420010222495284

In [81]:
ypre = search3.predict(dft)

In [83]:
ypre.shape
my_df = pd.DataFrame(ypre)
my_df.to_csv('sgd1.csv',index= True)